**Problem 3**

In [2]:

# Useful additional packages 
import matplotlib.pyplot as plt
import numpy as np
from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
import torch




"""
q = QuantumRegister(2)

qc = QuantumCircuit(q)
qc.u(pi,0,pi,q[0])
qc.draw()
"""


#lets create GHZ state
circ = QuantumCircuit(3)
# Add a H gate on qubit 0, putting this qubit in superposition.
circ.h(0)
# Add a CX (CNOT) gate on control qubit 0 and target qubit 1, putting
# the qubits in a Bell state.
circ.cx(0, 1)
# Add a CX (CNOT) gate on control qubit 0 and target qubit 2, putting
# the qubits in a GHZ state.
circ.cx(0, 2)
# Set the intial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2**3)
state1 = state.copy()
# Evolve the state by the quantum circuit
state = state.evolve(circ)
print(state)
print(state1)

Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))
Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [3]:
#function that apply the KS test to two probability list
def KS(P1, P2):
    assert len(P1) == len(P2)
    cdf1 = [P1[0]]
    cdf2 = [P2[0]]
    for i in range(len(P1)-1):
        cdf1.append(cdf1[i] + P1[i+1])
        cdf2.append(cdf2[i] + P2[i+1])
    difference = torch.tensor(cdf1) - torch.tensor(cdf2)

    #print(difference)
    return torch.pow(difference, 2).sum()#difference.abs().max().item()

print(KS([0.2, 0.8], [0.5, 0.5]))

tensor(0.0900)


In [4]:
from torch.nn import functional as F
import torch.nn as nn

#This is our KS network duh
class KS_net(nn.Module):
    def __init__(self):
        super(KS_net, self).__init__()
        self.linear1 = nn.Linear(9, 20)
        self.linear2 = nn.Linear(20, 20)
        self.linear3 = nn.Linear(20, 1)
    
    def forward(self, param):
        x = F.relu(self.linear1(param))
        #print(x)
        x = F.relu(self.linear2(x))
        #print(x)
        x = nn.Sigmoid()(self.linear3(x))
        return x

In [ ]:
#this is where we try to learn the symmetry
import random

class SymFinder():
  def __init__(self, eta, step_size):
    self.parameters = torch.randint(0, 100, (3, 3)) * pi /100
    #parameters[i,0] is the theta for ith qubit, 1 is \phi, 2 is lambda
    self.original_state = state
    self.transformed_state = None
    self.param_rate = eta # this is the learning rate for updating parameter
    self.step_size = step_size
    self.losses = []
    self.known_symmetries = []

    #now these are the nn attributes
    self.data = torch.zeros(30000, 10) #data for training KS_net, 9 + 1 = param + value
    self.data.requires_grad = False
    self.learn_rate = 0.01 #learning rate for training KS_net
    self.model = KS_net()
    self.batch_size = 20 #batch for SGD
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learn_rate)
    self.loss_func = nn.MSELoss()
    self.memory_pointer = 0 #tell where to store new data
    self.valid_range = 0#how much valid data is in the tensor, dont want to learn 00000

    #these are attributes for putting the nn and KS together
    self.reg_constant = 0.4
    self.KS_threshold = 0.0001 #if less than value, declare it as symmetry
    #self.model2 = KS_net()
    #self.model2.load_state_dict(self.model.state_dict())

  
  #return a transformed state according to parameter
  def transform(self, p):
    q = QuantumRegister(3)
    qc = QuantumCircuit(q)
    qc.u(p[0,0].item(),p[0,1].item(),p[0,2].item(),q[0])
    qc.u(p[1,0].item(),p[1,1].item(),p[1,2].item(),q[1])
    qc.u(p[2,0].item(),p[2,1].item(),p[2,2].item(),q[2])
    return self.original_state.copy().evolve(qc)


  
  def change_basis(self, state1, state2):
    q = QuantumRegister(3)
    qc = QuantumCircuit(q)
    qc.u(pi/2, 0, 0, q[0])
    qc.u(pi/2, 0, 0, q[1])
    qc.u(pi/2, 0, 0, q[2])
    return state1.copy().evolve(qc), state2.copy().evolve(qc)

  #return the loss from KS test of original vs another state
  def calculate_loss(self, state2, param):
    #get the probability in the original basis
    P1 = self.original_state.probabilities()
    P2 = state2.probabilities()
    #now we calculate probability in another basis
    new_state1, new_state2 = self.change_basis(self.original_state, state2)
    Q1 = new_state1.probabilities()
    Q2 = new_state2.probabilities()
    #lets add regularizer from our nn
    v = self.model(param.view(9)).detach_()
    #v = 0
    return [KS(P1, P2) + 0 * KS(Q1, Q2) - self.reg_constant * v, KS(P1, P2) + 0 * KS(Q1, Q2)]
  

  def update_KS_net(self):
    #this function use the data gathered to update KS_net parameters
    for i in range(30):
      self.optimizer.zero_grad()
      num = min(self.valid_range, self.data.size(0))
      indices = random.sample(range(num), self.batch_size)
      sampled_data = self.data[indices].clone()
      target = sampled_data[:, 9] #actual KS_value
      current = self.model(sampled_data[:, 0:9]).squeeze()
      loss = self.loss_func(current, target)
      self.losses.append(loss.item())
      #print(loss)
      loss.backward()
      self.optimizer.step()
    
  def update_param(self):
    #calculate the gradient using good old finite difference:
    cur_state = self.transform(self.parameters)
    cur_loss = self.calculate_loss(cur_state, self.parameters)[0]
    cur_true_loss = self.calculate_loss(cur_state, self.parameters)[1]
    #lets store this data
    self.data[self.memory_pointer, 0:9] = self.parameters.reshape(9)
    self.data[self.memory_pointer, 9] = cur_true_loss
    self.memory_pointer = (self.memory_pointer + 1) % self.data.size(0)
    self.valid_range = self.valid_range + 1
    grad = torch.zeros(3,3)
    for i in range(3):
        for j in range(3):
            new_param = self.parameters.clone()
            new_param[i,j] = new_param[i,j] + self.step_size
            new_state = self.transform(new_param)
            new_loss = self.calculate_loss(new_state, new_param)[0]
            grad[i,j] = (new_loss - cur_loss) / self.step_size
    #update the parameters:
    self.parameters = self.parameters - self.param_rate * grad
    #self.parameters.requires_grad = False
    #self.parameters[:, 2] = torch.zeros(3)
    return cur_true_loss
  



  def symfinding(self):
    self.parameters = torch.randint(0, 100, (3, 3)) * 2 * pi /100
    for i in range(700):
      if self.update_param() < self.KS_threshold:
        self.known_symmetries.append(self.parameters.clone())
        break



  def train(self):
    for i in range(8):
      self.symfinding()
      self.update_KS_net()
      print(i)
      #self.model2.load_state_dict(self.model.state_dict())


  def current_matrix(self):
    p = self.parameters
    #p[0,:] = torch.tensor([3.14159, 0,3.1415926])
    u0 = torch.tensor([[torch.cos(p[0,0]/2), -torch.exp(-p[0,2]*1j)*torch.sin(p[0,0]/2)],\
                    [torch.exp(p[0,1]*1j)*torch.sin(p[0,0]/2), torch.exp((p[0,1] + p[0,2])*1j)*torch.cos(p[0,0]/2)]])
    u1 = torch.tensor([[torch.cos(p[1,0]/2), -torch.exp(-p[1,2]*1j)*torch.sin(p[1,0]/2)],\
                    [torch.exp(p[1,1]*1j)*torch.sin(p[1,0]/2), torch.exp((p[1,1] + p[1,2])*1j)*torch.cos(p[1,0]/2)]])
    u2 = torch.tensor([[torch.cos(p[2,0]/2), -torch.exp(-p[2,2]*1j)*torch.sin(p[2,0]/2)],\
                    [torch.exp(p[2,1]*1j)*torch.sin(p[2,0]/2), torch.exp((p[2,1] + p[2,2])*1j)*torch.cos(p[2,0]/2)]])
    return [u0, u1, u2]
        
        
        
        

import numpy as np

finder = SymFinder(0.05, 0.01)
finder.train()
plt.plot(range(len(finder.losses)), finder.losses)

plt.show()



  

  
  
  

  


0
1
2
3
4


In [ ]:
plt.figure(figsize=(20, 3))

plt.plot(range(len(finder.losses)), np.log10(finder.losses))
#plt.plot(range(len(finder.losses)), finder.losses)

plt.show()




for i in range(10):
    parameters = torch.randint(0, 100, (3, 3)) * 2 * pi /100
    #parameters = finder.data[i, 0: 9].reshape(3,3)
    #trained_value = finder.data[i, 9]
    state2 = finder.transform(parameters)
    #print(parameters)
    print(finder.calculate_loss(state2, parameters)[1])
    print(finder.model(parameters.view(9)).detach_())
    print("---")



def matrix(p):
    #p[0,:] = torch.tensor([3.14159, 0,3.1415926])
    u0 = torch.tensor([[torch.cos(p[0,0]/2), -torch.exp(-p[0,2]*1j)*torch.sin(p[0,0]/2)],\
                    [torch.exp(p[0,1]*1j)*torch.sin(p[0,0]/2), torch.exp((p[0,1] + p[0,2])*1j)*torch.cos(p[0,0]/2)]])
    u1 = torch.tensor([[torch.cos(p[1,0]/2), -torch.exp(-p[1,2]*1j)*torch.sin(p[1,0]/2)],\
                    [torch.exp(p[1,1]*1j)*torch.sin(p[1,0]/2), torch.exp((p[1,1] + p[1,2])*1j)*torch.cos(p[1,0]/2)]])
    u2 = torch.tensor([[torch.cos(p[2,0]/2), -torch.exp(-p[2,2]*1j)*torch.sin(p[2,0]/2)],\
                    [torch.exp(p[2,1]*1j)*torch.sin(p[2,0]/2), torch.exp((p[2,1] + p[2,2])*1j)*torch.cos(p[2,0]/2)]])
    return [u0, u1, u2]

for p in finder.known_symmetries:
    print(matrix(p)[0])
    print(matrix(p)[1])
    print(matrix(p)[2])
    print("---")






In [ ]:
print(torch.exp(torch.tensor([3+3j]))